![cupy](resources/cupy.png)

# What is CuPy?

A NumPy-compatible array library accelerated by CUDA

### Top features:
- open source [https://github.com/cupy/cupy](https://github.com/cupy/cupy)
- high performance with cuda
- highly compatible with numpy
- easy to install
- easy to write a custom kernel
- docs [https://docs.cupy.dev/en/stable/](https://docs.cupy.dev/en/stable/)
- MIT License

In [ ]:
import os, sys
from time import time
import numpy as np
import cupy as cp

In [ ]:
class Timer(object):
    def __init__(self, description=None):
        self.description = description
        self.steps = {}
    def __call__( self, step_name=None):
        if step_name not in self.steps.keys():
            self.steps[step_name] = 0
        return self
    def __enter__(self):
        self.start = time()
    def __exit__(self, type, value, traceback):
        self.end = time()
        val = self.end - self.start
        key = self._last_key()
        if key:
            self.steps[key] += val
        print(f"{val}")
    def _last_key(self):
        return list(self.steps.keys())[-1] if len(self.steps.keys()) > 0 else None
    @property
    def total(self):
        return sum(list(self.steps.values()))

# Basics

## API:
- compatible GPU alternative of `numpy.ndarray` \
  `x_gpu = cp.array([1, 2, 3])` vs. `x_cpu = np.array([1, 2, 3])`
- most of the array manipulations are also done \
  `cp.linalg.norm(x_gpu)` vs. `np.linalg.norm(x_cpu)`
- Reference: [https://docs.cupy.dev/en/stable/reference/index.html#cupy-reference](https://docs.cupy.dev/en/stable/reference/index.html#cupy-reference)

In [ ]:
x_gpu0 = cp.array([1, 2, 3, 4, 5])
arr = cp.vstack([x_gpu0]*5)
print('array:\n', arr)
print('det:', cp.linalg.det(arr))

### CuPy has a concept of the *current device*

In [ ]:
x_gpu0 = cp.array([1, 2, 3, 4, 5])
print(f'x_gpu0 device: {x_gpu0.device}\n')

cp.cuda.Device(1).use()
x_gpu1 = cp.array([1, 2, 3, 4, 5])

with cp.cuda.Device(1):
    x_gpu1 = cp.array([1, 2, 3, 4, 5])

with cp.cuda.Device(0):
    x_gpu0 * 2

with cp.cuda.Device(1):
    try:
        x_gpu0 * 2 # error
    except:
        print(f'{sys.exc_info()[0].__name__}: {sys.exc_info()[1]}')

## Copy mem: *host* --> *device*

In [ ]:
x_cpu = np.array([1, 2, 3])
x_gpu = cp.asarray(x_cpu)

## Copy mem: *device* --> *host*

In [ ]:
tmp1 = cp.asnumpy(x_gpu)
# or
tmp2 = x_gpu.get()

### Memory

CuPy uses memory pool for memory allocations:
- Device memory pool (GPU device memory), which is used for GPU memory allocations.
- Pinned memory pool (non-swappable CPU memory), which is used during CPU-to-GPU data transfer.

In [ ]:
# Device 1
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()
print(f'Used memory on device 1 [bytes]: {mempool.used_bytes()}')
# print(f'Pinned memory [bytes]: {pinned_mempool.}')

# Kernels

- Elementwise
- Reduction
- Raw <--

### Add vectors

In [ ]:
timer = Timer('add_vec')
vec_size = int(5e7)

# Create random GPU vectors
with timer("GPU_create_random_vectors"):
    vec_a_gpu = cp.random.random(vec_size, dtype=cp.float64)
    vec_b_gpu = cp.random.random(vec_size, dtype=cp.float64)
    vec_c_gpu = cp.empty(vec_size, dtype=np.float64)
with timer("CPU_create_random_vectors"):
    vec_a_cpu = np.random.random(vec_size)
    vec_b_cpu = np.random.random(vec_size)
print(f'Used mem: {vec_a_gpu.nbytes/1024**3*3} GB')

### Elementwise kernel

In [ ]:
add_vec_elem = cp.ElementwiseKernel(
    'float64 x, float64 y',
    'float64 z',
    'z = x + y',
    'add_vec_elem')

#### Execute

In [ ]:
with timer("GPU_add_vectors"):
    vec_c_gpu = add_vec_elem(vec_a_gpu, vec_b_gpu)
with timer("CPU_add_vectors"):
    vec_c_cpu = np.add(vec_a_cpu, vec_b_cpu)

In [ ]:
timer.steps

### Clear memory

In [ ]:
# CPU
del vec_a_cpu
del vec_b_cpu
del vec_c_cpu

In [ ]:
# GPU: When the array goes out of scope, the allocated device memory is released and kept in the pool for future reuse.
del vec_a_gpu
del vec_b_gpu
del vec_c_gpu
print(f'Used [bytes]: {mempool.used_bytes()}')
print(f'Allocated [bytes]: {mempool.total_bytes()}')

In [ ]:
# clear the memory pool
mempool.free_all_blocks()

print(f'Allocated [bytes]: {mempool.total_bytes()}')

# Something big :)

### Creat random 3D array in GPU RAM (select GPU)

In [ ]:
batch = tuple([1024]*3)
cp.cuda.Device(0).use()
mempool = cp.get_default_memory_pool()
timer = Timer('smth_big')

with timer("GPU_create_random_array"):
    x_gpu = cp.random.random(batch)
print(f'Used [GB]: {mempool.used_bytes()/1024**3}')

In [ ]:
with timer("CPU_create_random_array"):
    x_cpu = np.random.random(batch)

In [ ]:
with timer("GPU_do_stuff"):
    x_gpu *= 3
    x_gpu *= x_gpu
with timer("CPU_do_stuff"):
    x_cpu *= 3
    x_gpu *= x_gpu

In [ ]:
del x_cpu
del x_gpu
mempool.free_all_blocks()
timer.steps

# Raw kernel

In [ ]:
timer = Timer('raw_kernel')
z_dim = 4096
yx_dims = (512, 512)
# layer = np.arange(1,XY**2+1, dtype=np.float).reshape(XY,XY)
with timer("CPU_create_array"):
    layer = np.random.rand(*yx_dims)
    arr = np.stack([layer]*z_dim, axis=0)

![gpu vs cpu](resources/cpu_vs_gpu.png)

![struct](resources/gpu.jpg)

### Device 0: "Quadro P6000"

| Param											| Value								|
| --- | --- |
| Total amount of global memory:                | 24450 MBytes (25637224448 bytes)								|
| (30) Multiprocessors, (128) CUDA Cores/MP:    | 3840 CUDA Cores												|
| GPU Max Clock rate:                           | 1645 MHz (1.64 GHz)											|
| Warp size:                                    | 32															|
| Maximum number of threads per multiprocessor: | 2048															|
| Maximum number of threads per block:          | 1024															|
| Max dimension size of a thread block (x,y,z): | (1024, 1024, 64)												|
| Max dimension size of a grid size    (x,y,z): | (2147483647, 65535, 65535)									|

In [ ]:
warp_size = 32

g_size = tuple((d//warp_size+int(bool(d%warp_size)) for d in yx_dims[::-1]))
b_size = (warp_size, warp_size)
print(f'Grid size: {g_size}\nBlock size: {b_size}')

Below you can find two raw kernels:
1. `mul_kernel` this kernel in the loop accumulate multiplication results straight into the output array. Output array is in global device memory, so access to this memory is slower than to the registry.
2. `mul_kernel_reg` this kernel is accumulating the multiplication results into the registry variable `tmp`. For each step of the loop the access is faster for this variable than to the global memory. This kernel should be 10 time faster than mul_kernel.


This example didn't show such time result during presentation because I changed the kernel code instead of creating two separate kernels (as it is right now).

In [ ]:
mul_kernel = cp.RawKernel(r'''
    extern "C" __global__
    void mul(const double* in, double* out, const int Z, const int Y, const int X) {
        int x = blockDim.x * blockIdx.x + threadIdx.x;
        int y = blockDim.y * blockIdx.y + threadIdx.y;
        if (x < X && y < Y) {
            double tmp = 1.;
            out[y*X+x] = 1.;
            for (int z = 0; z < Z; z++)
                out[y*X+x] = out[y*X+x] * in[z*Y*X+y*X+x];
            //out[y*X+x] = tmp;
        }
    }
    ''', 'mul')
mul_kernel.compile()

mul_kernel_reg = cp.RawKernel(r'''
    extern "C" __global__
    void mulReg(const double* in, double* out, const int Z, const int Y, const int X) {
        int x = blockDim.x * blockIdx.x + threadIdx.x;
        int y = blockDim.y * blockIdx.y + threadIdx.y;
        if (x < X && y < Y) {
            double tmp = 1.;
            for (int z = 0; z < Z; z++)
                tmp = tmp * in[z*Y*X+y*X+x];
            out[y*X+x] = tmp;
        }
    }
    ''', 'mulReg')
mul_kernel_reg.compile()

In [ ]:
dev = 0
with timer("copy host-->dev"):
    in_arr = cp.asarray(arr, dtype=cp.double)

In [ ]:
out_arr_reg = cp.empty(yx_dims, dtype=cp.double)

with timer("GPU__create_empty_output_array"):
    out_arr = cp.empty(yx_dims, dtype=cp.double)

with timer("GPU_calc_mul"):
    mul_kernel(g_size, b_size, (in_arr, out_arr, *in_arr.shape))
with Timer():
    mul_kernel_reg(g_size, b_size, (in_arr, out_arr_reg, *in_arr.shape))

In [ ]:
with timer("copy dev-->host"):
    out_gpu = cp.asnumpy(out_arr)

In [ ]:
with timer("CPU_calc"):
    out_cpu = np.ones_like(layer)
    for l in arr[:]:
        out_cpu *= l

In [ ]:
np.testing.assert_array_almost_equal(out_gpu, out_cpu)

In [ ]:
timer.steps

- Support for `__cuda_array_interface__`\
  The *cuda array interface* allows for interoperability between different implementation of GPU array-like objects in various projects\
  [https://numba.pydata.org/numba-doc/dev/cuda/cuda_array_interface.html](https://numba.pydata.org/numba-doc/dev/cuda/cuda_array_interface.html)
- TensorFlow doesn't support it yet, but there is a hope\
  [https://github.com/tensorflow/tensorflow/issues/29039](https://github.com/tensorflow/tensorflow/issues/29039)